In [1]:
import json
import numpy as np
import os
import glob
from tqdm.auto import tqdm, trange
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchtext
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
train_data = pd.read_csv('raw_data/court_texts.csv', names=['idx', 'text1', 'target', 'text2', 'case_number', 'text3', 'tokenized'])
train_data.tokenized = train_data.tokenized.apply(eval)
train_data.head()

,idx,text1,target,text2,case_number,text3,tokenized
0,105000,\n20 арбитражный апелляционный суд\nТимашкова ...,19,"</span></b></p>\r\n<p style=""text-align:center...",А54-4312/2008,"</span></b></p>\r\n<p style=""text-align:center...","[общество, ограничить, ответственность, ORG, д..."
1,105001,\n20 арбитражный апелляционный суд\nНикулова М...,3,\n20 арбитражный апелляционный суд\nНикулова М...,А62-6708/2008,\n20 арбитражный апелляционный суд\nНикулова М...,"[арбитражный, апелляционный, суд, никулов, м, ..."
2,105002,\n20 арбитражный апелляционный суд\nСтаханова ...,36,"</span></b></p>\r\n<p style=""text-align:center...",А23-1432/2008,"</span></b></p>\r\n<p style=""text-align:center...","[открытый, акционерный, общество, ORG, далее, ..."
3,105003,\n20 арбитражный апелляционный суд\nКапустина ...,6,\n20 арбитражный апелляционный суд\nКапустина ...,А23-2543/2009,\n20 арбитражный апелляционный суд\nКапустина ...,"[арбитражный, апелляционный, суд, капустин, ла..."
4,105004,\n20 арбитражный апелляционный суд\nТучкова Ол...,3,"</span></b></p>\r\n<p style=""text-align:center...",А62-2328/2009,"</span></b></p>\r\n<p style=""text-align:center...","[закрытый, акционерный, общество, ORG, далее, ..."


In [20]:
(train_data.tokenized.map(len) < 160).mean()

0.7096258919774281

In [4]:
cats = {
    "16": "Корпоративные споры",
    "36": "Споры по делам об оспаривании нормативных правовых актов в области таможенного дела",
    "3": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам купли-продажи",
    "35": "Споры по делам об оспаривании зарегистрированных прав на недвижимое имущество и сделок с ним",
    "21": "Споры о привлечении к административной ответственности",
    "1": "Споры о заключении договоров(контрактов)",
    "28": "Споры по делам об оспаривании ненормативных правовых актов Правительства РФ",
    "25": "Споры по введению процедур банкротства",
    "19": "Споры по делам об оспаривании решений налоговых органов о привлечении к административной ответственности",
    "32": "Споры о возмещении вреда в связи с обеспечением иска",
    "22": "Споры о признании права собственности",
    "26": "Споры об оспаривании решений третейских судов",
    "5": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам аренды",
    "2": "Споры по искам антимонопольных органов об оспаривании нормативных правовых актов",
    "6": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам подряда",
    "4": "Споры о признании договоров недействительными",
    "27": "Споры по делам об оспаривании ненормативных правовых актов Президента РФ",
    "20": "Споры о ненадлежащем исполнении и возмещении убытков",
    "17": "Споры, связанные с созданием, реорганизацией и ликвидацией юридических лиц",
    "24": "Споры об обжаловании решений Роспатента",
    "10": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам займа и кредита",
    "9": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам в сфере транспортной деятельности",
    "37": "Споры об изъятии, прекращении или ограничении права на земельный участок",
    "14": "Споры о неисполнении или ненадлежащем исполнении обязательств по посредническим договорам",
    "12": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам хранения",
    "11": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам банковского счета, при осуществлении расчетов",
    "13": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам возмездного оказания услуг",
    "15": "Споры о неисполнении или ненадлежащем исполнении обязательств по иным видам договоров",
    "33": "Споры о создании, реорганизации и ликвидации организаций",
    "18": "Споры по делам об оспаривании ненормативных правовых актов, решений и действий (бездействия) государственных внебюджетных органов",
    "23": "Споры, связанные с охраной интеллектуальной собственности",
    "34": "Споры об уклонении от государственной регистрации юридических лиц и индивидуальных предпринимателей",
    "30": "Нет",
    "7": "Споры по искам антимонопольных органов об изменении или расторжении договора",
    "0": "Споры, возникающие в связи с неисполнением или ненадлежащим исполнением обязательств из совершения с землей сделок купли-продажи",
    "8": "Споры о неисполнении или ненадлежащем исполнении обязательств по договорам долевого участия в строительстве",
    "29": "Экономические споры между субъектами Российской Федерации"
}
train_data['target_descr'] = [cats[str(t)] for t in train_data.target]
train_data.head()

,idx,text1,target,text2,case_number,text3,tokenized,target_descr
0,105000,\n20 арбитражный апелляционный суд\nТимашкова ...,19,"</span></b></p>\r\n<p style=""text-align:center...",А54-4312/2008,"</span></b></p>\r\n<p style=""text-align:center...","[общество, ограничить, ответственность, ORG, д...",Споры по делам об оспаривании решений налоговы...
1,105001,\n20 арбитражный апелляционный суд\nНикулова М...,3,\n20 арбитражный апелляционный суд\nНикулова М...,А62-6708/2008,\n20 арбитражный апелляционный суд\nНикулова М...,"[арбитражный, апелляционный, суд, никулов, м, ...",Споры о неисполнении или ненадлежащем исполнен...
2,105002,\n20 арбитражный апелляционный суд\nСтаханова ...,36,"</span></b></p>\r\n<p style=""text-align:center...",А23-1432/2008,"</span></b></p>\r\n<p style=""text-align:center...","[открытый, акционерный, общество, ORG, далее, ...",Споры по делам об оспаривании нормативных прав...
3,105003,\n20 арбитражный апелляционный суд\nКапустина ...,6,\n20 арбитражный апелляционный суд\nКапустина ...,А23-2543/2009,\n20 арбитражный апелляционный суд\nКапустина ...,"[арбитражный, апелляционный, суд, капустин, ла...",Споры о неисполнении или ненадлежащем исполнен...
4,105004,\n20 арбитражный апелляционный суд\nТучкова Ол...,3,"</span></b></p>\r\n<p style=""text-align:center...",А62-2328/2009,"</span></b></p>\r\n<p style=""text-align:center...","[закрытый, акционерный, общество, ORG, далее, ...",Споры о неисполнении или ненадлежащем исполнен...


In [21]:
train_data.target.value_counts()

3     21706
25    19687
21    12906
36    11643
14     7850
6      6427
12     6374
19     5849
5      5837
2      4584
26     4143
28     4136
32     3728
22     3370
37     2413
4      2007
16     1924
9      1859
17     1842
10     1498
35      812
1       715
33      444
24      396
34      259
0       252
13      198
15      193
18      176
20      156
27      140
8       139
23      130
11       88
7        55
30       36
Name: target, dtype: int64

In [5]:
cat_sizes = train_data.target_descr.value_counts()
cat_sizes

Споры о неисполнении или ненадлежащем исполнении обязательств по договорам купли-продажи                                             21706
Споры по введению процедур банкротства                                                                                               19687
Споры о привлечении к административной ответственности                                                                               12906
Споры по делам об оспаривании нормативных правовых актов в области таможенного дела                                                  11643
Споры о неисполнении или ненадлежащем исполнении обязательств по посредническим договорам                                             7850
Споры о неисполнении или ненадлежащем исполнении обязательств по договорам подряда                                                    6427
Споры о неисполнении или ненадлежащем исполнении обязательств по договорам хранения                                                   6374
Споры по делам об оспариван

# Reproduce tokenization

In [43]:
import re
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    NamesExtractor,
    AddrExtractor,
    MoneyExtractor,
    DatesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)

def replace_date(x, token='DATE'):
    date_extracts = list(dates_extractor(x))
    if len(date_extracts) == 0:
        return x
    else:
        return_text = x
        for date_extract in date_extracts: 
            st_idx = date_extract.start
            end_idx = date_extract.stop            
            return_text = return_text.replace(x[st_idx:end_idx], token)
        return return_text

def replace_address(x, token='ADDR'):
    addr_extracts = list(addr_extractor(x))
    if len(addr_extracts) == 0:
        return x
    else:
        return_text = x
        for addr_extract in addr_extracts: 
            if addr_extract.fact.type in ['улица', 'дом']:
                st_idx = addr_extract.start
                end_idx = addr_extract.stop            
                return_text = return_text.replace(x[st_idx:end_idx], token)
        return return_text

def replace_money(x, quantiles=None):
    # text = judgment_w_motive.motiv_part.iloc[0]
    money_extracts = list(money_extractor(x))
    if len(money_extracts) == 0:
        return x
    else:
        return_text = x
        for money_extract in money_extracts:
            print(money_extract)
            st_idx = money_extract.start
            end_idx = money_extract.stop
            currency = money_extract.fact.currency
            if currency != 'RUB':
                return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_F')
                continue
            amount = money_extract.fact.amount
            if amount < 4631.79:
                print('money_0')
                return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_0')
                continue
            if amount <= 168830.91:
                return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_1')
                continue
            return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_2')
            
        return return_text

def money_in_span(x):
    for money_token in ['MONEY_F', 'MONEY_0', 'MONEY_1', 'MONEY_2', 'DATE']:
        if money_token in x:
            return True
    return False

def replace_org(x, leave_org = None):
    doc = Doc(x)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    return_text = x
    for span in doc.spans:
        if span.type == 'ORG':
            if ((leave_org is not None) and (span.text in leave_org)) or money_in_span(span.text):
                continue
            else:
                word = 'ORG'
                return_text = return_text.replace(span.text, word)
    return return_text

org_freq = pd.read_csv('org_freq.csv')
leave_org = (org_freq[org_freq.freq > 65].org_name.values).tolist()

def prepare_russian_text(raw_text):
    doc = Doc(raw_text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    doc.tag_morph(morph_tagger)

    prepared_text = []
    for token in doc.tokens:
        if token.text in ['ORG', 'DATE', 'MONEY_0', 'MONEY_1', 'MONEY_2']:
            prepared_text.append(token.text)
            continue
        skip_pos = ['PUNCT', 'ADP', 'SCONJ', 'CCONJ', 'SYM', 'NUM']
        if token.pos not in skip_pos:
            try:
                token.lemmatize(morph_vocab)
                prepared_text.append(token.lemma.lower())
            except Exception as ex:
                prepared_text.append(token.text.lower())
    return prepared_text

def pipeline(text: str):
    try:
        text = re.sub('<(?:"[^"]*"[\'"]*|\'[^\']*\'[\'"]*|[^\'">])+>', '', text.replace('&nbsp;', ''))
        text_dates_removed = replace_date(text, '')
        # text_addr_removed = replace_address(text_dates_removed)
        # text_money_replaced = replace_money(text_addr_removed)
        text_money_replaced = replace_money(text_dates_removed)
        text_money_org_replaced = replace_org(text_money_replaced, leave_org)
        lemmatised_text = prepare_russian_text(text_money_org_replaced)
        return lemmatised_text
    except:
        return text

In [14]:
re.sub('<(?:"[^"]*"[\'"]*|\'[^\']*\'[\'"]*|[^\'">])+>', '', train_data.text3[0])

'\r\n&nbsp;\r\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 общество с ограниченной ответственностью «Ярмарка» (далее – ООО «Ярмарка», Общество) обратилось в Арбитражный суд Рязанской области с заявлением о признании незаконным бездействия Администрации г.Рязани, выраженное в нерассмотрении заявления о выдаче разрешения на размещение и эксплуатацию временного сооружения №769-ВС4/08 от 11.07.2008, поданного ООО «Ярмарка» в установленный срок, как несоответствующее требованиям Положения о временных сооружениях на территории г.Рязани, утвержденного решением Рязанского городского Совета от 26.04.2007 №309-III (далее – Положение о временных сооружениях), в качестве устранения нарушения обязать Администрацию г.Рязани перезаключить договор совместного использования объекта недвижимости под временным строением, расположенным по адресу: г.Рязань, ул. Горького, д.1(с учетом уточнения).\r\nРешением Арбитражного суда Рязанской области от 17.02.2009 '

In [ ]:
i = 250
list(zip(pipeline(train_data.text3[i]), train_data.tokenized[i]))

# Lemmatised concat

In [ ]:
import pandas as pd

train_data = pd.read_csv('court_texts.csv', names=['idx', 'text1', 'target', 'text2', 'case_number', 'text3', 'tokenized'])

In [ ]:
from sklearn.model_selection import train_test_split

new_train_data = pd.read_csv('train_trans.csv')
new_val_data = pd.read_csv('val_trans.csv')

def pipeline(text: str):
    try:
        text = ' '.join(text.split())
        text = text.replace('&quot;', '"')
        return text
    except:
        return text

texts_train, texts_val, targets_train, targets_val = train_test_split(
    train_data.tokenized.map(lambda s: ' '.join(eval(s))), train_data.target,
    test_size=0.1, stratify=train_data.target, random_state=123
)
train_mask = list(new_train_data.text.map(pipeline).str.len() > 32)
pd.DataFrame({'text': texts_train.reset_index(drop=True)[train_mask], 'target': targets_train.reset_index(drop=True)[train_mask]}).to_csv('train_trans_lem.csv', index=False)
val_mask = list(new_val_data.text.map(pipeline).str.len() > 32)
pd.DataFrame({'text': texts_val.reset_index(drop=True)[val_mask], 'target': targets_val.reset_index(drop=True)[val_mask]}).to_csv('val_trans_lem.csv', index=False)

# Vocab for CNN

In [4]:
np.quantile(list(map(len, train_data['tokenized'])), 0.97)

522.0

In [5]:
vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data['tokenized'], specials=['<pad>', '<unk>'], min_freq=10,
)

In [6]:
max_length = 512

train_tokens = []
for text in tqdm(train_data['tokenized']):
    tokens = [vocab[word] if word in vocab else vocab['<unk>'] for word in text]
    train_tokens += [tokens]

tokenized_train = torch.full((len(train_tokens), max_length), vocab['<pad>'], dtype=torch.int32)
for i, tokens in tqdm(enumerate(train_tokens)):
    length = min(max_length, len(tokens))
    tokenized_train[i, :length] = torch.tensor(tokens[:length])

  0%|          | 0/133972 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [4]:
# tokenized_train, tokenized_val, targets_train, targets_val = train_test_split(
#     tokenized_train, train_data.target, test_size=0.1, stratify=train_data.target, random_state=123
# )
tokenized_train, tokenized_val, targets_train, targets_val = train_test_split(
    train_data.tokenized, train_data.target, test_size=0.1, stratify=train_data.target, random_state=123
)
tokenized_train.map(lambda l: ' '.join(l)).to_csv('interpreting-cnn-for-text/legal_dataset/train.txt.tok', header=False, index=False)
tokenized_val.map(lambda l: ' '.join(l)).to_csv('interpreting-cnn-for-text/legal_dataset/test.txt.tok', header=False, index=False)
targets_train.to_csv('interpreting-cnn-for-text/legal_dataset/train.cat', header=False, index=False)
targets_val.to_csv('interpreting-cnn-for-text/legal_dataset/test.cat', header=False, index=False)

In [13]:
train_dataset = torch.utils.data.TensorDataset(tokenized_train, torch.tensor(targets_train.to_list()))
val_dataset = torch.utils.data.TensorDataset(tokenized_val, torch.tensor(targets_val.to_list()))

batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Models

In [14]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout_proba):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout_proba)
        
    def forward(self, x):                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

# Preprocessing for transformer

In [2]:
train_data = pd.read_csv('train_trans_trim.csv')
texts_train, targets_train = train_data.text, train_data.target
val_data = pd.read_csv('val_trans_trim.csv')
texts_val, targets_val = val_data.text, val_data.target

In [5]:
import re
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    NamesExtractor,
    AddrExtractor,
    MoneyExtractor,
    DatesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)

def replace_date(x, token='DATE'):
    date_extracts = list(dates_extractor(x))
    if len(date_extracts) == 0:
        return x
    else:
        return_text = x
        for date_extract in date_extracts: 
            st_idx = date_extract.start
            end_idx = date_extract.stop            
            return_text = return_text.replace(x[st_idx:end_idx], token)
        return return_text

def replace_address(x, token='ADDR'):
    addr_extracts = list(addr_extractor(x))
    if len(addr_extracts) == 0:
        return x
    else:
        return_text = x
        for addr_extract in addr_extracts: 
            if addr_extract.fact.type in ['улица', 'дом']:
                st_idx = addr_extract.start
                end_idx = addr_extract.stop            
                return_text = return_text.replace(x[st_idx:end_idx], token)
        return return_text

def replace_money(x, quantiles=None):
    money_extracts = list(money_extractor(x))
    if len(money_extracts) == 0:
        return x
    else:
        return_text = x
        for money_extract in money_extracts:
            st_idx = money_extract.start
            end_idx = money_extract.stop
            currency = money_extract.fact.currency
            if currency != 'RUB':
                return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_F')
                continue
            amount = money_extract.fact.amount
            if amount < 4631.79:
                return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_0')
                continue
            if amount <= 168830.91:
                return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_1')
                continue
            return_text = return_text.replace(x[st_idx:end_idx], 'MONEY_2')
            
        return return_text

def money_in_span(x):
    for money_token in ['MONEY_F', 'MONEY_0', 'MONEY_1', 'MONEY_2', 'DATE']:
        if money_token in x:
            return True
    return False

def replace_org(x, leave_org = None):
    doc = Doc(x)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    return_text = x
    for span in doc.spans:
        if span.type == 'ORG':
            if ((leave_org is not None) and (span.text in leave_org)) or money_in_span(span.text):
                continue
            else:
                word = 'ORG'
                return_text = return_text.replace(span.text, word)
    return return_text

org_freq = pd.read_csv('org_freq.csv')
leave_org = (org_freq[org_freq.freq > 65].org_name.values).tolist()

def prepare_russian_text(raw_text):
    text = raw_text.lower()
    text = text.replace('org', 'ORG').replace('money_', 'MONEY_')
    return text

In [6]:
train_data = pd.read_csv('train_trans.csv')
texts_train, targets_train = train_data.text, train_data.target
val_data = pd.read_csv('val_trans.csv')
texts_val, targets_val = val_data.text, val_data.target

from pandarallel import pandarallel
pandarallel.initialize()

def pipeline(text: str):
    try:
        text_dates_removed = replace_date(' '.join(text.split()), '')
        # text_addr_replaced = replace_address(text_dates_removed)
        # text_money_replaced = replace_money(text_addr_replaced)
        text_money_replaced = replace_money(text_dates_removed)
        text_money_org_replaced = replace_org(text_money_replaced, leave_org)
        return prepare_russian_text(text_money_org_replaced)
    except:
        return text


pd.DataFrame({'text': texts_train.parallel_map(pipeline), 'target': targets_train}).to_csv('train_trans_DMO_2.csv')
pd.DataFrame({'text': texts_val.parallel_map(pipeline), 'target': targets_val}).to_csv('val_trans_DMO_2.csv')

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [12]:
from sklearn.model_selection import train_test_split

new_train_data = pd.read_csv('train_trans.csv')
new_val_data = pd.read_csv('val_trans.csv')

def pipeline(text: str):
    try:
        text = ' '.join(text.split())
        text = text.replace('&quot;', '"')
        return text
    except:
        return text

# train_data = pd.read_csv('train_trans_DMO_2.csv')
# texts_train, targets_train = train_data.text, train_data.target
texts_train = pd.read_csv('interpreting-cnn-for-text/legal_dataset/train.txt.tok', header=None)
targets_train = pd.read_csv('interpreting-cnn-for-text/legal_dataset/train.cat', header=None)

# train_mask = list(new_train_data.text.map(pipeline).str.len() > 32)
# pd.DataFrame({'text': texts_train.reset_index(drop=True)[train_mask], 'target': targets_train.reset_index(drop=True)[train_mask]}).to_csv('train_trans_DMO_3.csv', index=False)
train_mask = list(new_train_data.text.map(pipeline).str.len() > 32)
texts_train[train_mask].to_csv('interpreting-cnn-for-text/legal_dataset/train.txt.tok', index=False, header=False)
targets_train[train_mask].to_csv('interpreting-cnn-for-text/legal_dataset/train.cat', index=False, header=False)

In [10]:
train_dataset[0][0].strip()

'рассматривается исковое заявление ОАО «Военно-страховая компания»\xa0\xa0 к ООО «Росгосстрах» о взыскании\xa0 120000 руб. 00 коп.\xa0 страхового возмещения.\r\nПредставители\xa0 ОАО «Военно-страховая компания»\xa0\xa0 и ООО «Росгосстрах» в судебное заседание не'

In [ ]:
from tokenizers.decoders import ByteLevel

tokens = train_collater.tokenizer.tokenize(train_dataset[0][0].strip())
for t in tokens:
    print(ByteLevel().decode(t))

# Train/val functions

In [3]:
from tqdm import tqdm
from IPython.display import clear_output

# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.CrossEntropyLoss()

def train_epoch(model, optimizer, sched, device):
    loss_log, acc_log = [], []
    model.train()
    for data, target in tqdm(train_loader):
        # data = data.to(device)
        for key, value in data.items():
            data[key] = value.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        # output = model(data)
        output = model(**data).logits

        # loss = loss_fn(output, F.one_hot(target, num_classes=38).float())
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        loss_log.append(loss.item())

        preds = torch.argmax(output, dim=-1)
        acc_log.append((preds == target).float().mean().item())

        if sched is not None:
            sched.step(loss_log[-1])
    return loss_log, acc_log  

def test(model, device):
    loss_log, acc_log = [], []
    model.eval()
    with torch.no_grad():
        for data, target in tqdm(val_loader):
            data = data.to(device)
            target = target.to(device)

            output = model(data)
            loss = loss_fn(output, F.one_hot(target, num_classes=38).float())
            loss_log.append(loss.item())

            preds = torch.argmax(output, dim=-1)
            acc_log.append((preds == target).float().mean().item())
    return loss_log, acc_log  

def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    plt.plot(train_history, label='train', zorder=1)    
    points = np.array(val_history)
    plt.scatter(points[:, 0], points[:, 1], marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    plt.legend(loc='best')
    plt.grid()
    plt.show()
    
def train(model, opt, sched, n_epochs, device, path_suffix=''):
    train_loss_log, train_acc_log = [], []
    val_loss_log, val_acc_log = [], []

    best_acc = 0.0
    for epoch in range(n_epochs):
        print("Epoch {0} of {1}".format(epoch, n_epochs))
        
        train_loss, train_acc = train_epoch(model, opt, sched, device)
        val_loss, val_acc = test(model, device)
        
        train_loss_log.extend(train_loss)
        train_acc_log.extend(train_acc)
        
        steps = len(train_dataset) / batch_size
        val_loss_log.append((steps * (epoch + 1), np.mean(val_loss)))
        val_acc_log.append((steps * (epoch + 1), np.mean(val_acc)))
        
        clear_output()
        plot_history(train_loss_log, val_loss_log)
        plot_history(train_acc_log, val_acc_log, title='accuracy')
        
        if val_acc_log[-1][1] > best_acc:
            best_acc = np.mean(val_acc)
            torch.save(model.state_dict(), f'./best_model{path_suffix}.pt')
        
        diff = val_acc_log[-1][1] - (val_acc_log[-2][1] if len(val_acc_log) > 1 else 0)
        print(f'Last val accuracy = {val_acc_log[-1][1]:.4f} ({diff:+.4f})')
        print(f'Best val accuracy = {best_acc:.4f}')
    
    model.load_state_dict(torch.load(f'./best_model{path_suffix}.pt'))

# Finally, run!

In [ ]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 512
N_FILTERS = 256
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 72
DROPOUT_PROBA = 0.5

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT_PROBA).to(device)
lr = 0.001
n_epochs = 10
opt = torch.optim.Adam(model.parameters(), lr)

train(model, opt, None, n_epochs, device)

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

gpu_id = 0
device = torch.device(f'cuda:{gpu_id}') if gpu_id is not None else torch.device('cpu')

model = AutoModelForSequenceClassification.from_pretrained(backbone_model_name, num_labels=38)

current_segment_embeddings = model.roberta.embeddings.token_type_embeddings.weight.data
segment_embeddings = torch.cat(
    [
        current_segment_embeddings,
        current_segment_embeddings + torch.rand_like(current_segment_embeddings) * 0.01
    ]
)
model.roberta.embeddings.token_type_embeddings = model.roberta.embeddings.token_type_embeddings.from_pretrained(
    segment_embeddings,
    freeze=False
)

model.to(device)

lr = 0.001
n_epochs = 10
opt = torch.optim.Adam(model.parameters(), lr)

train(model, opt, None, n_epochs, device)

In [40]:
len(train_dataset), len(val_dataset), len(cat_sizes)

(120574, 13398, 36)

In [41]:
pd.DataFrame({'text': train_dataset.texts, 'target': train_dataset.targets}).to_csv('train_trans.csv')
pd.DataFrame({'text': val_dataset.texts, 'target': val_dataset.targets}).to_csv('val_trans.csv')